## VQE procedure in qiskit and tequila

#### Qiskit

In [5]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit import Parameter
import numpy as np

In [6]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(atom=f"H 0 0 0; H 0 0 1", spin=0, basis="sto-3g")
mol = driver.run()

mapper = JordanWignerMapper()

initial_state = HartreeFock( mol.num_spatial_orbitals, mol.num_particles, mapper)

ansatz = UCCSD( mol.num_spatial_orbitals, mol.num_particles, 
                mapper, initial_state=initial_state)

In [7]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

vqe = VQE(Estimator(), ansatz, SLSQP()) # here a custom ansatz can be used instead of the inbuilt qiskit one

# making shure that qubits are set to 0
vqe.initial_point = np.zeros(ansatz.num_parameters)
    
solver = GroundStateEigensolver(mapper, vqe)
result = solver.solve(mol)

result.total_energies[0]

-1.1011503188066496

### Tequila

In [8]:
import tequila as tq

samples = 1000000
backend = "qiskit"
device = "fake_rome"

# define the molecule
molecule = tq.chemistry.Molecule(geometry = "H 0.0 0.0 0.0\nH 0.0 0.0 7",
                                 basis_set="sto-3g",
                                 transformation="jordan_wigner")

fci = molecule.compute_energy("fci")

H = molecule.make_hamiltonian()


U = tq.gates.Ry(angle="a", target=0)
U += tq.gates.X(target=1, control=0)
E = tq.ExpectationValue(H=H, U=U, optimize_measurements=True)

vqe = tq.minimize(method="cobyla", objective=E, initial_values=0.0)
noisy_vqe = tq.minimize(method="cobyla", objective=E, samples=samples, backend=backend, device=device, initial_values=0.0)

# UpCCGSD
UpCCGSD = molecule.make_upccgsd_ansatz(name="UCCD") # string can be modified to specify the ansatz
E2 = tq.ExpectationValue(H=H, U=UpCCGSD, optimize_measurements=True)
ucc = tq.minimize(method="cobyla", objective=E2, initial_values=0.0)
noisy_ucc = tq.minimize(method="cobyla", objective=E2, samples=samples,  backend=backend, device=device, initial_values=0.0)

print("VQE         = {:2.8f}".format(min(vqe.history.energies)))
print("VQE (noisy) = {:2.8f}".format(min(noisy_vqe.history.energies)))
print("UCC         = {:2.8f}".format(min(ucc.history.energies)))
print("UCC (noisy) = {:2.8f}".format(min(noisy_ucc.history.energies)))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qiskit
device          : None
samples         : None
save_history    : True
noise           : None

Method          : COBYLA
Objective       : 2 expectationvalues

active variables : 1

E=+0.07559674  angles= {a: 0.0}  samples= None
E=-0.07593253  angles= {a: 1.0}  samples= None
E=-0.39120504  angles= {a: 2.0}  samples= None
E=-0.58036069  angles= {a: 3.0}  samples= None
E=-0.46949065  angles= {a: 4.0}  samples= None
E=-0.56271378  angles= {a: 3.5}  samples= None
E=-0.55870738  angles= {a: 2.75}  samples= None
E=-0.58361407  angles= {a: 3.125}  samples= None
E=-0.58172442  angles= {a: 3.25}  samples= None
E=-0.58331216  angles= {a: 3.1875}  samples= None
E=-0.58328227  angles= {a: 3.09375}  samples= None
E=-0.58365929  angles= {a: 3.140625}  samples= None
E=-0.58362404  angles= {a: 3.15625}  samples= None
E=-0.58365172  angles= {a: 3.1484375}  samples= None
E=-0.58365553  angles= {a: 3.13671875}  